In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
match_df_clean = pd.read_csv('match_clean.csv')
team_df = pd.read_csv('team.csv', index_col='team_api_id')
transfers_in = pd.read_csv('transfers_in_clean.csv', index_col=0)
transfers_out = pd.read_csv('transfers_out_clean.csv', index_col=0)
spending_by_year_finish = pd.read_csv('spending_by_year_finish.csv', index_col=0)

In [54]:
transfers_in_by_year = transfers_in.groupby(['club_api_id','club_name','year','league_name',
                                             'age_range','position']).agg({'age':'mean',
                                                                           'fee_cleaned_millions':'sum',
                                                                           'transfer_movement':'count'})
transfers_in_by_year.rename(columns={'age':'average_age', 
                                     'fee_cleaned_millions':'total_spent', 
                                     'transfer_movement':'nTransfers'}, inplace=True)
transfers_in_by_year.reset_index(level=['year','league_name','age_range','position'], inplace=True)

transfers_out_by_year = transfers_out.groupby(['club_api_id','club_name','year','league_name',
                                               'age_range','position']).agg({'age':'mean',
                                                                             'fee_cleaned_millions':'sum',
                                                                             'transfer_movement':'count'})
transfers_out_by_year.rename(columns={'age':'average_age', 
                                     'fee_cleaned_millions':'total_player_revenue', 
                                     'transfer_movement':'nTransfers'}, inplace=True)
transfers_out_by_year.reset_index(level=['year','league_name','age_range','position'], inplace=True)

In [55]:
years_in_top_division = transfers_in_by_year.groupby(['club_api_id','club_name'])['year'].nunique()
top_division_clubs = years_in_top_division[years_in_top_division == 10].index.tolist()
top_division_clubs = [top_division_clubs[i][0] for i in range(len(top_division_clubs))]
transfers_in_by_year = transfers_in_by_year.loc[top_division_clubs]
transfers_out_by_year = transfers_out_by_year.loc[top_division_clubs]

In [131]:
#returns dataframe of end of season table for given year, league
def get_league_table(season, league_name):
    season_df = match_df_clean[(match_df_clean['league_name']==league_name) & (match_df_clean['season']==season)]
    
    league_table_df = pd.DataFrame(0, columns=['team_id','Wins', 'Draws', 'Losses', 'Goals_For', 'Goals_Against', 'Goal_Difference', 'Points'],
                                   index=season_df['home_team_name'].unique()).rename_axis('Team')
        
    for i, row in season_df.iterrows():
        league_table_df.loc[row['home_team_name'], 'Goals_For'] += row['home_team_goal']
        league_table_df.loc[row['home_team_name'], 'Goals_Against'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_For'] += row['away_team_goal']
        league_table_df.loc[row['away_team_name'], 'Goals_Against'] += row['home_team_goal']
        
        if row['home_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Wins'] += 1
            league_table_df.loc[row['away_team_name'], 'Losses'] += 1
        elif row['away_win'] == True:
            league_table_df.loc[row['home_team_name'], 'Losses'] += 1
            league_table_df.loc[row['away_team_name'], 'Wins'] += 1
        else:
            league_table_df.loc[row['home_team_name'], 'Draws'] += 1
            league_table_df.loc[row['away_team_name'], 'Draws'] += 1
        
    league_table_df['Goal_Difference'] = league_table_df['Goals_For'] - league_table_df['Goals_Against']
    league_table_df['Points'] = league_table_df['Wins']*3 + league_table_df['Draws']
    league_table_df['Win %'] = league_table_df['Wins'] / (league_table_df['Wins']+league_table_df['Draws']+league_table_df['Losses'])
    league_table_df = league_table_df.sort_values(['Points', 'Goal_Difference'], ascending=False).reset_index()
    league_table_df.index = pd.RangeIndex(start=1, stop=len(league_table_df)+1, step=1)
    pos_val_list = [i for i in np.arange(100,0,step=-5)]
    league_table_df['Position_Value'] = pos_val_list[:len(league_table_df)]
    for i, row in league_table_df.iterrows():
        league_table_df.loc[i, 'team_id'] = team_df[team_df.team_long_name==row['Team']].index[0]
    return league_table_df

#returns league finish of given team at end of given season
def get_league_finish(season, team_id):
    league = match_df_clean.league_name[match_df_clean.home_team_api_id==team_id].tolist()[0]
    league_table = get_league_table(season, league)
    team_name = team_df.loc[team_id, 'team_long_name']
    finish = league_table[league_table['Team'] == team_name].index.values.astype(int)[0]
    return finish

In [132]:
get_league_table(2009, 'Primera Division')

,Team,team_id,Wins,Draws,Losses,Goals_For,Goals_Against,Goal_Difference,Points,Win %,Position_Value
1,FC Barcelona,8634,31,6,1,98,24,74,99,0.815789,100
2,Real Madrid,8633,31,3,4,102,35,67,96,0.815789,95
3,Valencia CF,10267,21,8,9,59,40,19,71,0.552632,90
4,Sevilla FC,8302,19,6,13,65,49,16,63,0.500000,85
5,RCD Mallorca,8661,18,8,12,59,44,15,62,0.473684,80
6,Getafe CF,8305,17,7,14,58,48,10,58,0.447368,75
7,Villarreal CF,10205,16,8,14,58,57,1,56,0.421053,70
8,Athletic Bilbao,8315,15,9,14,50,53,-3,54,0.394737,65
9,Atlético Madrid,9906,13,8,17,57,61,-4,47,0.342105,60
10,RC Deportivo de La Coruña,9783,13,8,17,35,49,-14,47,0.342105,55


In [56]:
transfers_in_by_year = transfers_in_by_year[(transfers_in_by_year.year != 2007) & (transfers_in_by_year.year != 2016)]
transfers_out_by_year = transfers_out_by_year[(transfers_out_by_year.year != 2007) & (transfers_out_by_year.year != 2016)]

In [57]:
transfers_in_by_year.head(10)

year league_name age_range    position  \
club_api_id club_name                                                      
6403        FC Paços de Ferreira  2008    Liga Nos     15-22    Defender   
            FC Paços de Ferreira  2008    Liga Nos     15-22     Forward   
            FC Paços de Ferreira  2008    Liga Nos     15-22  Midfielder   
            FC Paços de Ferreira  2008    Liga Nos     23-29    Defender   
            FC Paços de Ferreira  2008    Liga Nos     23-29     Forward   
            FC Paços de Ferreira  2008    Liga Nos     23-29  Midfielder   
            FC Paços de Ferreira  2008    Liga Nos       30+    Defender   
            FC Paços de Ferreira  2008    Liga Nos       30+  Midfielder   
            FC Paços de Ferreira  2009    Liga Nos     15-22    Defender   
            FC Paços de Ferreira  2009    Liga Nos     15-22     Forward   

                                  average_age  total_spent  nTransfers  
club_api_id club_name                                                   
6403        FC Paços de Ferreira    21.000000     0.000000           1  
            FC Paços de Ferreira    20.333333     0.000000           3  
            FC Paços de Ferreira    19.500000     0.000596           2  
            FC Paços de Ferreira    26.285714     0.002430           7  
            FC Paços de Ferreira    26.250000     0.001215           4  
            FC Paços de Ferreira    23.333333     0.001215           3  
            FC Paços de Ferreira    30.000000     0.000000           2  
            FC Paços de Ferreira    33.000000     0.000000           1  
            FC Paços de Ferreira    20.000000     0.000000           2  
            FC Paços de Ferreira    21.000000     0.001192           6

In [171]:
transfer_stats_df = transfers_in_by_year.groupby(['club_api_id','club_name','year','league_name']).agg({'average_age':'mean',
                                                                                    'total_spent':'sum',
                                                                                    'nTransfers':'sum'}).reset_index(level=3)

In [61]:
for i, row in transfer_stats_df.iterrows():
    tdf = transfers_in_by_year.loc[i[:2]]
    transfer_stats_df.loc[i, '15-22'] = tdf[(tdf.year==i[2]) & (tdf.age_range=='15-22')]['total_spent'].sum()
    transfer_stats_df.loc[i, '23-29'] = tdf[(tdf.year==i[2]) & (tdf.age_range=='23-29')]['total_spent'].sum()
    transfer_stats_df.loc[i, '30+'] = tdf[(tdf.year==i[2]) & (tdf.age_range=='30+')]['total_spent'].sum()
    transfer_stats_df.loc[i, 'Defender'] = tdf[(tdf.year==i[2]) & (tdf.position=='Defender')]['total_spent'].sum()
    transfer_stats_df.loc[i, 'Forward'] = tdf[(tdf.year==i[2]) & (tdf.position=='Forward')]['total_spent'].sum()
    transfer_stats_df.loc[i, 'Midfielder'] = tdf[(tdf.year==i[2]) & (tdf.position=='Midfielder')]['total_spent'].sum()

In [62]:
transfer_stats_df.head()

league_name  average_age  total_spent  \
club_api_id club_name            year                                         
6403        FC Paços de Ferreira 2008    Liga Nos    24.962798     0.005456   
                                 2009    Liga Nos    22.744444     0.004837   
                                 2010    Liga Nos    23.708333     0.060811   
                                 2011    Liga Nos    23.792007     0.014558   
                                 2012    Liga Nos    22.206349     0.004814   

                                       nTransfers     15-22     23-29  30+  \
club_api_id club_name            year                                        
6403        FC Paços de Ferreira 2008          23  0.000596  0.004860  0.0   
                                 2009          27  0.001192  0.003645  0.0   
                                 2010          19  0.059596  0.001215  0.0   
                                 2011          35  0.001192  0.013366  0.0   
                                 2012          26  0.002384  0.002430  0.0   

                                       Defender   Forward  Midfielder  \
club_api_id club_name            year                                   
6403        FC Paços de Ferreira 2008  0.002430  0.001215    0.001811   
                                 2009  0.001215  0.002407    0.001215   
                                 2010  0.001215  0.059596    0.000000   
                                 2011  0.004241  0.007886    0.002430   
                                 2012  0.003026  0.000000    0.001788   

                                       league_position  position_value  \
club_api_id club_name            year                                    
6403        FC Paços de Ferreira 2008             10.0            55.0   
                                 2009              9.0            60.0   
                                 2010              7.0            70.0   
                                 2011             10.0            55.0   
                                 2012              3.0            90.0   

                                        win_pct  goals_for  goals_against  \
club_api_id club_name            year                                       
6403        FC Paços de Ferreira 2008  0.300000       37.0           42.0   
                                 2009  0.266667       32.0           37.0   
                                 2010  0.333333       35.0           42.0   
                                 2011  0.266667       35.0           53.0   
                                 2012  0.466667       42.0           29.0   

                                       goal_difference  three_year_spending  \
club_api_id club_name            year                                         
6403        FC Paços de Ferreira 2008             -5.0                  NaN   
                                 2009             -5.0                  NaN   
                                 2010             -7.0             0.071104   
                                 2011            -18.0             0.080206   
                                 2012             13.0             0.080182   

                                       two_year_spending  pos_val_pct_change  
club_api_id club_name            year                                         
6403        FC Paços de Ferreira 2008                NaN                 NaN  
                                 2009           0.010293            9.090909  
                                 2010           0.065648           16.666667  
                                 2011           0.075369          -21.428571  
                                 2012           0.019371           63.636364

In [186]:
for i, row in transfer_stats_df.iterrows():
    league_df = get_league_table(i[2], row.league_name)
    relevant_stats = league_df[league_df.team_id==i[0]]
    transfer_stats_df.loc[i, 'league_position'] = relevant_stats.index.astype(int)
    transfer_stats_df.loc[i, 'position_value'] = relevant_stats.Position_Value.tolist()[0]
    transfer_stats_df.loc[i, 'win_pct'] = relevant_stats['Win %'].tolist()[0]
    transfer_stats_df.loc[i, 'goals_for'] = relevant_stats.Goals_For.tolist()[0]
    transfer_stats_df.loc[i, 'goals_against'] = relevant_stats.Goals_Against.tolist()[0]
    transfer_stats_df.loc[i, 'goal_difference'] = relevant_stats.Goal_Difference.tolist()[0]

In [63]:
transfer_stats_df.reset_index(inplace=True)

In [65]:
transfer_stats_df = pd.read_csv('transfer_stats.csv', index_col=0)
transfer_stats_df.head()

,club_api_id,club_name,year,league_name,average_age,total_spent,nTransfers,15-22,23-29,30+,...,Midfielder,league_position,position_value,win_pct,goals_for,goals_against,goal_difference,three_year_spending,two_year_spending,pos_val_pct_change
0,6403,FC Paços de Ferreira,2008,Liga Nos,24.962798,0.005456,23,0.000596,0.004860,0.0,...,0.001811,10.0,55.0,0.300000,37.0,42.0,-5.0,NaN,NaN,NaN
1,6403,FC Paços de Ferreira,2009,Liga Nos,22.744444,0.004837,27,0.001192,0.003645,0.0,...,0.001215,9.0,60.0,0.266667,32.0,37.0,-5.0,NaN,0.010293,9.090909
2,6403,FC Paços de Ferreira,2010,Liga Nos,23.708333,0.060811,19,0.059596,0.001215,0.0,...,0.000000,7.0,70.0,0.333333,35.0,42.0,-7.0,0.071104,0.065648,16.666667
3,6403,FC Paços de Ferreira,2011,Liga Nos,23.792007,0.014558,35,0.001192,0.013366,0.0,...,0.002430,10.0,55.0,0.266667,35.0,53.0,-18.0,0.080206,0.075369,-21.428571
4,6403,FC Paços de Ferreira,2012,Liga Nos,22.206349,0.004814,26,0.002384,0.002430,0.0,...,0.001788,3.0,90.0,0.466667,42.0,29.0,13.0,0.080182,0.019371,63.636364


In [60]:
transfer_stats_df.set_index(['club_api_id','club_name','year'], inplace=True)
transfer_stats_df

league_name  average_age  \
club_api_id club_name            year                                  
6403        FC Paços de Ferreira 2008          Liga Nos    24.962798   
                                 2009          Liga Nos    22.744444   
                                 2010          Liga Nos    23.708333   
                                 2011          Liga Nos    23.792007   
                                 2012          Liga Nos    22.206349   
...                                                 ...          ...   
10267       Valencia CF          2011  Primera Division    23.875000   
                                 2012  Primera Division    26.366667   
                                 2013  Primera Division    25.729167   
                                 2014  Primera Division    24.397619   
                                 2015  Primera Division    23.460714   

                                       total_spent  nTransfers  15-22  23-29  \
club_api_id club_name            year                                          
6403        FC Paços de Ferreira 2008     0.005456          23    6.0   14.0   
                                 2009     0.004837          27   14.0   13.0   
                                 2010     0.060811          19   12.0    5.0   
                                 2011     0.014558          35   18.0   15.0   
                                 2012     0.004814          26   15.0   11.0   
...                                            ...         ...    ...    ...   
10267       Valencia CF          2011    30.600000          16   10.0    5.0   
                                 2012    20.880000          10    2.0    7.0   
                                 2013    20.228000          15    6.0    7.0   
                                 2014    47.970000          23    9.0   13.0   
                                 2015   129.510000          20    8.0   12.0   

                                       30+  Defender  Forward  Midfielder  \
club_api_id club_name            year                                       
6403        FC Paços de Ferreira 2008  3.0      10.0      7.0         6.0   
                                 2009  0.0       7.0     11.0         9.0   
                                 2010  2.0       8.0      7.0         4.0   
                                 2011  2.0      12.0     15.0         8.0   
                                 2012  0.0      12.0      8.0         6.0   
...                                    ...       ...      ...         ...   
10267       Valencia CF          2011  1.0       7.0      4.0         5.0   
                                 2012  1.0       3.0      1.0         6.0   
                                 2013  2.0       3.0      7.0         5.0   
                                 2014  1.0       8.0      6.0         9.0   
                                 2015  0.0       8.0      9.0         3.0   

                                       league_position  position_value  \
club_api_id club_name            year                                    
6403        FC Paços de Ferreira 2008             10.0            55.0   
                                 2009              9.0            60.0   
                                 2010              7.0            70.0   
                                 2011             10.0            55.0   
                                 2012              3.0            90.0   
...                                                ...             ...   
10267       Valencia CF          2011              3.0            90.0   
                                 2012              5.0            80.0   
                                 2013              8.0            65.0   
                                 2014              4.0            85.0   
                                 2015             11.0            50.0   

                                        win_pct  goals_for  goals_against  \
club_api_id club_name   

In [31]:
for i in range(2010, 2016):
    for index, row in transfer_stats_df.iterrows():
        if index[2] == i:
            transfer_stats_df.loc[index, 'three_year_spending'] = transfer_stats_df.loc[index, 'total_spent']+transfer_stats_df.loc[(index[0],index[1],index[2]-1), 'total_spent']+transfer_stats_df.loc[(index[0],index[1],index[2]-2), 'total_spent']

for i in range(2009, 2016):
    for index, row in transfer_stats_df.iterrows():
        if index[2] == i:
            transfer_stats_df.loc[index, 'two_year_spending'] = transfer_stats_df.loc[index, 'total_spent']+transfer_stats_df.loc[(index[0],index[1],index[2]-1), 'total_spent']

In [42]:
transfer_stats_df.set_index(['club_api_id','club_name'], inplace=True)

In [48]:
for i, row in transfer_stats_df.iterrows():
    if i == 0:
        transfer_stats_df.loc[i, 'pos_val_pct_change'] = np.nan
    elif i > 0:
        if transfer_stats_df.loc[i, 'club_name'] == transfer_stats_df.loc[i-1, 'club_name']:
            transfer_stats_df.loc[i, 'pos_val_pct_change'] = (transfer_stats_df.loc[i, 'position_value'] - transfer_stats_df.loc[i-1, 'position_value'])*100/transfer_stats_df.loc[i-1, 'position_value']
        elif transfer_stats_df.loc[i, 'club_name'] != transfer_stats_df.loc[i-1, 'club_name']:
            transfer_stats_df.loc[i, 'pos_val_pct_change'] = np.nan

In [67]:
transfer_stats_df.head(20)

,club_api_id,club_name,year,league_name,average_age,total_spent,nTransfers,15_22,23_29,Over_30,...,Midfielder,league_position,position_value,win_pct,goals_for,goals_against,goal_difference,three_year_spending,two_year_spending,pos_val_pct_change
0,6403,FC Paços de Ferreira,2008,Liga Nos,24.962798,0.005456,23,0.000596,0.004860,0.0,...,0.001811,10.0,55.0,0.300000,37.0,42.0,-5.0,NaN,NaN,NaN
1,6403,FC Paços de Ferreira,2009,Liga Nos,22.744444,0.004837,27,0.001192,0.003645,0.0,...,0.001215,9.0,60.0,0.266667,32.0,37.0,-5.0,NaN,0.010293,9.090909
2,6403,FC Paços de Ferreira,2010,Liga Nos,23.708333,0.060811,19,0.059596,0.001215,0.0,...,0.000000,7.0,70.0,0.333333,35.0,42.0,-7.0,0.071104,0.065648,16.666667
3,6403,FC Paços de Ferreira,2011,Liga Nos,23.792007,0.014558,35,0.001192,0.013366,0.0,...,0.002430,10.0,55.0,0.266667,35.0,53.0,-18.0,0.080206,0.075369,-21.428571
4,6403,FC Paços de Ferreira,2012,Liga Nos,22.206349,0.004814,26,0.002384,0.002430,0.0,...,0.001788,3.0,90.0,0.466667,42.0,29.0,13.0,0.080182,0.019371,63.636364
5,6403,FC Paços de Ferreira,2013,Liga Nos,24.153061,0.120886,31,0.000596,0.120290,0.0,...,0.002430,16.0,25.0,0.200000,28.0,59.0,-31.0,0.140258,0.125700,-72.222222
6,6403,FC Paços de Ferreira,2014,Liga Nos,23.766327,0.002430,27,0.000000,0.002430,0.0,...,0.000000,8.0,65.0,0.352941,40.0,45.0,-5.0,0.128130,0.123316,160.000000
7,6403,FC Paços de Ferreira,2015,Liga Nos,23.347222,0.001788,23,0.001788,0.000000,0.0,...,0.000596,7.0,70.0,0.382353,43.0,42.0,1.0,0.125104,0.004218,7.692308
8,7844,Vitória Guimarães SC,2008,Liga Nos,23.852381,0.733431,14,0.350783,0.382648,0.0,...,0.000000,8.0,65.0,0.333333,32.0,36.0,-4.0,NaN,NaN,NaN
9,7844,Vitória Guimarães SC,2009,Liga Nos,24.391667,0.646997,21,0.107349,0.539648,0.0,...,0.386566,6.0,75.0,0.366667,31.0,34.0,-3.0,NaN,1.380428,15.384615


In [69]:
transfer_stats_df.rename(columns={'15_22':'Fifteen_TwentyTwo','23_29':'TwentyThree_TwentyNine','Over_30':'Thirty_Plus'}, inplace=True)

In [46]:
transfer_stats_df.reset_index(inplace=True)

In [70]:
transfer_stats_df.to_csv('transfer_stats.csv')